In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import os
import shutil
import logging
import subprocess
from copy import deepcopy
from time import time
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdDistGeom import EmbedMultipleConfs
from linux_qm.src.util import _load_smiles3D, _create_tmp_dir, draw3Dconfs, SetPositions
from linux_qm.qm.crest.crest import conformer_pipeline


In [11]:
smi = 'COC1CN(C)C1'
# smi = 'CO'


# mol = _load_smiles3D(smi)
# EmbedMultipleConfs(mol, numConfs=3)

mol = conformer_pipeline(smi)

# Get the conformers

conformers = mol.GetConformers()

# Print the number of conformers generated
print(f"Number of conformers generated: {len(conformers)}")
print("Energies:", [c.GetProp('energy') for c in conformers])

[24-11-2023 01:22:05] [DEBUG] Success SMILES parsing
normal termination of xtb
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
[24-11-2023 01:22:06] [DEBUG] Success geometry optimization
[24-11-2023 01:22:09] [DEBUG] Success conformer generation
[24-11-2023 01:22:09] [DEBUG] Success conformer screen


Number of conformers generated: 4
Energies: ['-23.275472740000001', '-23.273513560000001', '-23.273084600000001', '-23.27281279']


In [12]:
query = Chem.MolFromSmarts('C1CCN1')
atom_ids = mol.GetSubstructMatch(query)
print(atom_ids)
AllChem.AlignMolConformers(mol, atomIds=atom_ids)
draw3Dconfs(mol, autoalign=False)

(3, 2, 6, 4)
num conformers 4


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [13]:
import os
os.getcwd()

'/home/ergot/linux_qm/notebooks/.orca_tmp/935a547a-1155-4dbd-8207-a8ee3ec7287b'

In [14]:
logging.getLogger().setLevel(logging.DEBUG)

In [15]:
from linux_qm.qm.orca.orca import OrcaDriver
orca = OrcaDriver()

conf = deepcopy(mol).GetConformer()

orca.options['method'] = 'HF def2-SVP'
orca.single_point(conf)

[24-11-2023 01:22:09] [DEBUG] ORCA INPUT:
!HF def2-SVP   
%geom MaxIter 100 end
%pal nprocs 16 end
%maxcore 2000
*xyz 0 1
C            2.71480243         -0.42883920         -0.15406392
O            1.61467542          0.39693376         -0.44576057
C            0.56782221          0.27295626          0.47466346
C           -0.36187959         -0.95003118          0.33957808
N           -1.51500067         -0.04688962          0.36976673
C           -2.58122719         -0.29505549         -0.56448760
C           -0.66125418          1.11991812          0.13117381
H            3.46300474         -0.22807853         -0.91811257
H            3.12947551         -0.19818496          0.83506987
H            2.43971257         -1.48950631         -0.18102984
H            0.93023177          0.41955975          1.50069697
H           -0.36202699         -1.68035882          1.15004077
H           -0.22255303         -1.45665493         -0.62353864
H           -2.23226242         -0.34935035   

In [ ]:
orca.geometry_optimization(conf)

[24-11-2023 01:22:13] [INFO] Geometry optimization
[24-11-2023 01:22:13] [INFO] Method: HF def2-SVP
[24-11-2023 01:22:13] [DEBUG] ORCA INPUT:
!HF def2-SVP OPT  
%geom MaxIter 100 end
%pal nprocs 16 end
%maxcore 2000
*xyz 0 1
C            2.71480243         -0.42883920         -0.15406392
O            1.61467542          0.39693376         -0.44576057
C            0.56782221          0.27295626          0.47466346
C           -0.36187959         -0.95003118          0.33957808
N           -1.51500067         -0.04688962          0.36976673
C           -2.58122719         -0.29505549         -0.56448760
C           -0.66125418          1.11991812          0.13117381
H            3.46300474         -0.22807853         -0.91811257
H            3.12947551         -0.19818496          0.83506987
H            2.43971257         -1.48950631         -0.18102984
H            0.93023177          0.41955975          1.50069697
H           -0.36202699         -1.68035882          1.15004077
H      

In [ ]:
%%time

logging.getLogger().setLevel(logging.INFO)
orca.options['solvent'] = 'THF'
orca.options['n_procs'] = 24

conf = deepcopy(mol).GetConformer()

orca.options['method'] = 'M062X 6-311++G(d,p) NMR'
orca.geometry_optimization(conf)

In [ ]:
%%time

conf = deepcopy(mol).GetConformer()

orca.options['method'] = 'M062X 6-31G'
orca.geometry_optimization(conf)

orca.options['method'] = 'M062X 6-311++G(d,p) NMR'
orca.geometry_optimization(conf)

In [ ]:
%%time

conf = deepcopy(mol).GetConformer()
orca.options['method'] = 'M062X cc-pVDZ NMR'
orca.geometry_optimization(conf)

In [ ]:
%%time
conf = deepcopy(mol).GetConformer()
orca.options['method'] = 'M062X cc-pVTZ NMR'
orca.geometry_optimization(conf)

In [ ]:
%%time
conf = deepcopy(mol).GetConformer()
orca.options['method'] = 'M062X aug-cc-pVTZ NMR'
orca.geometry_optimization(conf)

In [ ]:
%%time

conf = deepcopy(mol).GetConformer()
orca.options['method'] = 'M062X cc-pVDZ'
orca.geometry_optimization(conf)

orca.options['method'] = 'M062X aug-cc-pVTZ OPT NMR'
orca.geometry_optimization(conf)

In [ ]:
query = Chem.MolFromSmarts('C1CCN1')
atom_ids = mol.GetSubstructMatch(query)
print(atom_ids)
AllChem.AlignMolConformers(mol, atomIds=atom_ids)
draw3Dconfs(mol, confIds=[0], autoalign=False)

In [ ]:
data.scfvalues

In [ ]:
data.scftargets

In [ ]:
# NMR shielding
for k,v in data.nmrtensors.items():
    print(mol.GetAtomWithIdx(k).GetSymbol(), k, v['isotropic'])

In [ ]:
dir(data)

In [ ]:
print(data.metadata['cpu_time'][0])

In [ ]:
len(data.converged_geometries)

In [ ]:
dir(data)

In [ ]:
data.optdone

In [ ]:
data.metadata

In [ ]:
data.new_geometries

In [ ]:
# HOMO LUMO
homo = data.moenergies[0][data.homos[0]]
lumo = data.moenergies[0][data.homos[0] + 1]
homo, lumo

In [ ]:
print(data.metadata['cpu_time'][0])

In [ ]:
data.natom, data.nmo

In [ ]:
data.metadata

In [ ]:
data.scfenergies